In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier

from settings import DATA_DIR
from utils import cv

In [2]:
# Read the text file into a dataframe
X = pd.read_csv(os.path.join(DATA_DIR, 'x_train.txt'), sep=' ', header=None).to_numpy()
y = pd.read_csv(os.path.join(DATA_DIR, 'y_train.txt'), header=None).to_numpy().T[0]
X_test = pd.read_csv(os.path.join(DATA_DIR, 'x_test.txt'), sep=' ', header=None).to_numpy()

# Read the shape of the data
print(X.shape, y.shape)
print(X_test.shape)

(5000, 500) (5000,)
(5000, 500)


In [3]:
model = RandomForestClassifier(max_depth=5)
k_folds = 3

scores = cv(X, y, model, k_folds)

Using 500 features, we properly classified 600 clients.
Using 500 features, we properly classified 596 clients.
Using 500 features, we properly classified 632 clients.


In [4]:
scores

[-94000, -94040, -93680]

### Effectiveness evaluation

The effectiveness
of your prediction will be assessed as follows:
- For each designated customer who actually took advantage of the offer, the company will pay you $€10$.
- For each variable used, you must pay $€200$ (the company bears the cost of obtaining information related
to individual variables).

### Notes
- deadline: 3rd June
- at least 5 strategies (feature selection/model) must be implemented
- CV necessary
- feature selection methods
- finding and finetuning standalone models
- ensemble methods
- self-learning 